In [24]:
from pathlib import Path
import json
import pandas as pd
import librosa

In [25]:
FUSA_SERVER="https://api.labacam.org"
API_KEY="zIkYk2VwBiC762O4yFSd"

ENDPOINT="predictions"
MODEL="PANN-tag-5s"

URI = f"{FUSA_SERVER}/{ENDPOINT}?model={MODEL}/"

print(URI)

https://api.labacam.org/predictions?model=PANN-tag-5s/


In [26]:
threshold = 0.01

In [27]:
def filter_predictions(categories_dict, threshold):
    filtered_dict = {}
    for key, values in categories_dict.items():
        values_list = []
        for value in values:
            if value['probability'] > threshold:
                values_list.append(value)
            filtered_dict.update({key:values_list})
    return filtered_dict

def calculate_metrics(filtered_predictions, audio_duration):
    df = pd.DataFrame([], columns  = ['event', 'duration_ratio', 'avg_acc'])
    class_tags, duration_ratio, avg_acc = [], [], []
    for class_tags in filtered_predictions:
        avg_acc = 0
        tags = filtered_predictions[class_tags]
        c = len(tags)
        if c > 0:
            for tag in tags:
                avg_acc += tag['probability']
            duration_ratio = (c * 5) / audio_duration
            avg_acc = avg_acc / c
            d = {'event' : class_tags, 'duration_ratio': round(duration_ratio, 3), 'avg_acc': round(avg_acc, 3)}
            output = pd.DataFrame([d])
            df = pd.concat([df, output], ignore_index=True)
    df = df.sort_values(by='duration_ratio', ascending=False).head(5)
    return df

def get_audio_duration(audio_path):
    return librosa.get_duration(filename=audio_path)

In [28]:
audio_name = "paisaje.sonoro.2.ACUS15522.mancilla.mp3"
response = !curl -s -X POST {URI} -H  "accept: application/json" -H  "X-Api-Key:{API_KEY}" -H  "Content-Type: multipart/form-data" -F "file=@{audio_name};type=audio/wav"

try:
    categories_dict = json.loads(response[0])['categories']
    filtered_predictions = filter_predictions(categories_dict, threshold)
    audio_duration = get_audio_duration(audio_name)
    classes_metrics = calculate_metrics(filtered_predictions, audio_duration)
    audio_path = Path(audio_name)
    classes_metrics['filename'] = str(Path(audio_path))
except Exception as e:
    print(e, audio_name)

In [29]:
classes_metrics

,event,duration_ratio,avg_acc,filename
7,music/music,0.840,0.789,paisaje.sonoro.2.ACUS15522.mancilla.mp3
2,animal/others,0.420,0.034,paisaje.sonoro.2.ACUS15522.mancilla.mp3
4,mechanical/air_conditioner,0.252,0.152,paisaje.sonoro.2.ACUS15522.mancilla.mp3
0,animal/bird,0.168,0.812,paisaje.sonoro.2.ACUS15522.mancilla.mp3
3,humans/others,0.168,0.016,paisaje.sonoro.2.ACUS15522.mancilla.mp3
